In [2]:
pip install numpy<2

/bin/bash: line 1: 2: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Imports 
import numpy as np
import pandas as pd
import csv
from bs4 import BeautifulSoup


# Code From Erik Lang
### This section will be used as a reference for building a loop to extract the necessary information from the .HTML files. 


In [ ]:
# Path to your local HTML file
# file_path = 'path/to/your/file.html'
# Erik's comment: this 2011 example html file was saved in the same folder as this notebook, so adjust the file path accordingly
file_path = 'Assets/NBA_Contract_Data/NBA_Contract_Extensions_2015.html'

# Open and read the local file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Parse the HTML content
soup = BeautifulSoup(content, 'html.parser')

# Find the <tbody> section
tbody = soup.find('tbody')  #Erik's comment: the data in each year's table is between <tbody> </tbody>

# If found, extract rows
if tbody:  #Erik's comment:  "if <tbody> </tbody> exists, perform the next operation"
    rows = tbody.find_all('tr')
    for row in rows:
        cells = row.find_all(['td', 'th'])  # include <th> just in case
        cell_texts = [cell.get_text(strip=True) for cell in cells]
        print(cell_texts)
else:
    print("No <tbody> found in the file.")

# HTML File Scraper Function
### Building a function from the provide code above
#### This function takes a year (ex. 11,12,...20) as an input and will extract the data from the corresponding file_path. The function then takes the data and appends each list(signing year + row of individual player data) to a list. This will allow us to loop through multiple .HTML files while also creating a new column that will save the year as a new feature to the data.

In [9]:
def scrape_HTML(year):
    #Let's start with our file path. The year will be inputed so we can specify which year
    #we want to read from before writing to our csv file
    #The use of the year variable will also allow us to add the year as a new column to our
    #data ensuring that we do not lose any precision when combining the 10 files into 1 CSV
    file_path = f'Assets/NBA_Contract_Data/NBA_Contract_Extensions_20{year}.html'

    #Opening and read the specified file
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    #turn our read in html file into a nice pot of soup
    soup = BeautifulSoup(content, 'html.parser')

    #The data we want to scrape can be found between <tbody> and </tbody>
    #The following code finds this part (if it exists) and grabs the data from the section
    #and will append each cell to our list of data
    tbody = soup.find('tbody')
    extracted_data = []

    
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            cells = row.find_all(['td','th'])
            cell_texts = [cell.get_text(strip=True) for cell in cells]
            #Now we can add our year variable so we make sure we don't lose that feature
            cell_texts.insert(0, f'20{year}')

            extracted_data.append(cell_texts)

    else:
        print('No <tbody> found in the file.')
    

    return extracted_data

# CSV Creation Function
### Now that we have our function to scrape a specified .HTML file, we can use it to create our combined CSV file. We are going to loop through a list of years, scrape the contract data from each HTML using our above function, and save everything into a single CSV file.

In [15]:
def create_contract_csv(output_file):
    #Start with our year list and some headers we can add (and ignore later) to help with readability 
    years = [11,12,13,14,15,16,17,18,19,20]
    headers = ['Signing Year', 'Rank', 'Player', 'Position','Team',
               'Age', 'Contract Length', 'Total Value', 'Avg. Value',
               'Guaranteed Money', 'Contract Type'] 

    #Creating an empty list that we can add each .html file data too after scraping 
    all_data = []
    for year in years:
        year_data = scrape_HTML(str(year))
        #Take our list  of lists from the function and extend it to our 
        all_data.extend(year_data)

    #Now that we have all of our data in 1 place we can write it to our csv file
    with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
        
        #create our writer object that will allow us to add to our output_file
        writer = csv.writer(csv_file)
        #start with adding our header information to the file
        writer.writerow(headers)

        for row in all_data:
            writer.writerow(row)

    print(f'Your CSV file {output_file} has been created and saved to the Assets Folder')
    #By default this function will save the CSV file to the same folder that this notebook
    #was run in unless we specify the Assets folder in out output_file variable
    return

In [16]:
#Now lets create our output_file path name and call our function
output_file_path = 'Assets/NBA_Contracts.csv'

create_contract_csv(output_file_path)

Your CSV file Assets/NBA_Contracts.csv has been created and saved to the Assets Folder
